# Introduction

This sample notebook takes you through an end-to-end workflow to demonstrate the functionality of SageMaker Ground Truth and Amazon Rekognition Custom Labels

In [ ]:
import datetime
import tarfile
import boto3
import os
from sagemaker import get_execution_role
import sagemaker
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont


## Upload Images to S3

In [ ]:
bucket_name = 'sagemaker-aiml' ## Updates the value with the bucket name created earlier in the lab

In [ ]:
## Uploading Licensed Images for raw data
region = boto3.Session().region_name    
source_dir = '../images/raw-data/LicensedImages-CreativeCommons'
dest_dir = 'raw-data/images'
file_list = os.listdir(source_dir)
s3_client = boto3.client('s3', region_name=region)
for file in file_list :   
    if file != '.ipynb_checkpoints':
        response = s3_client.upload_file(source_dir+'/'+file, bucket_name, dest_dir+"/"+file)
        print (file + ' uploaded')
print('Raw Data Upload Complete to '+bucket_name+'/'+dest_dir)

## Uploading Non-Licensed Images for raw data
source_dir = '../images/raw-data/LicenseNotNeeded_Images'
dest_dir = 'raw-data/images'
file_list = os.listdir(source_dir)
s3_client = boto3.client('s3', region_name=region)
for file in file_list : 
    if file != '.ipynb_checkpoints':
        response = s3_client.upload_file(source_dir+'/'+file, bucket_name, dest_dir+"/"+file)
        print (file + ' uploaded')
print('Raw Data Upload Complete to '+bucket_name+'/'+dest_dir)



## Uploading Test Data
source_dir = '../images/test-data'
dest_dir = 'test-data/images'
file_list = os.listdir(source_dir)
s3_client = boto3.client('s3', region_name=region)
for file in file_list : 
    response = s3_client.upload_file(source_dir+'/'+file, bucket_name, dest_dir+"/"+file)
    print (file + ' uploaded')
print('Test Data Upload Complete to '+bucket_name+'/'+dest_dir)


### Let's look at one of the images

In [ ]:
imageName = "raw-data/images/800px-Woodpeckers-Telephone-Cable.jpg"
display(IImage(url=s3_client.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': imageName})))

## Detect Object using Amazon Rekogniton
#### Attach IAM Managed Policy 
- Click on the generated URL
- Click on **Attach policies** button
- Search for **Rekog** in **Filter policies** bar
- Select **AmazonRekognitionFullAccess** and click on **Attach Policy**

In [ ]:
role_name = get_execution_role().split('/')[1]
job_url = "https://console.aws.amazon.com/iam/home?#/roles/"+role_name+"?section=permissions"
print (job_url) 

<img src="../lab-images/15.png">
<img src="../lab-images/16.png">

#### Assume Role
- Click on the generated URL
- Click on **Edit trust relationship** button
- Edit Policy with Service as **["sagemaker.amazonaws.com","rekognition.amazonaws.com"]**
- Click on **Update Trust Policy** button

In [ ]:
job_url = "https://console.aws.amazon.com/iam/home?#/roles/"+role_name+"?section=trust"
print (job_url) 

<img src="../lab-images/17.png" width="600">
<img src="../lab-images/18.png" width="600">

### Lets look at Object Detection

In [ ]:
# Call Amazon Rekognition to detect objects in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectLabels.html
imageName = "raw-data/images/4278289454_d4bcb08484_o.jpg"

# Init clients
rekognition = boto3.client('rekognition')

detectLabelsResponse = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': imageName,
        }
    }
)

imageName = "raw-data/images/4278289454_d4bcb08484_o.jpg"
display(IImage(url=s3_client.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': imageName})))

##Display list of Objects
print("Detected object:")
for label in detectLabelsResponse["Labels"]:
        print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))

As you can notice in the above response that Amazon Rekognition service detected objects in the provided image but did not detect the holes as an object of interest. This means we need to use Amazon Rekognition Custom Labels to detect custom labels in the image. Typically, you would need to identify the correct machine learning algorithm, machine learning model, tune the model and perform hyperparameter tuning. All of this is handled for you with just few lines of code in Amazon Rekognition Custom Labels. Let's see it in action.

# Ground Truth labeling job





Part or all of your images will be annotated by human annotators. It is essential to provide good instructions that help the annotators give you the annotations you want. Good instructions are:

Concise. We recommend limiting verbal/textual instruction to two sentences, and focusing on clear visuals.
Visual. In the case of image classification, we recommend providing one labeled image in each of the classes as part of the instruction.
When used through the AWS Console, Ground Truth helps you create the instructions using a visual wizard.

### Create Labeling Workforce
- Select **'Labeling workforces'** then click the **'Private'** tab.
- On the **'Private'** tab click **'Create private team'**

![labeling workforce](../lab-images/1a.png)

On the **'Create private team'** page
- Enter the **'Team name'** as **Labeling-experts**
- Click **Create private team**

![Create private team](../lab-images/1b.png)

Select **Invite new workers**
![Create private team](../lab-images/1c.png)

On the **Add workers by email address** page 
- Add your **email address** to invite private annotators to access the job. For the purpose of this exercise, you can use your own email address. Typically, this will be the list of email addresses of workers in your organization.
- Click **Invite new workers**

![Create private team](../lab-images/1d.png)

### Ground Truth Label Job
In the left hand menu select **Labeling Job**
![Create Labeling Job](../lab-images/1.png)

Click **Create labeling job**
![Label Job](../lab-images/2.png)

### Specify Job Parameters
- Specify Job Name - **'aws-workshops-woodpecker-holes'**
- Check the box next to "I want to specify a label attribute name different from the labeling job name."
- Specify a value of **'labels'** in the "Label attribute name" field
- Under "Input data setup" select "**Automated data setup**"
- For "S3 location for input datasets" specify the S3 location of images - **'s3://{your-bucket-name}/raw-data/images/'**
- Next select "Specify a new location" under "S3 location for output datasets" and specify the output location for annotated data - **'s3://{your-bucket-name}/annotated-data/'**
- For "Data type" select "images"

**Note:** When you see **{your-bucket-name}** replace it with the name of the bucket that you created earlier

<img src="../lab-images/labelingJob.png" width="700">

### Create IAM Role
- Select the option to **create a new role**
- Specify S3 Bucket Name - **'{your-bucket-name}'** 
- Click on **Create** button

### Complete Data setup
- Click on "**Complete Data Setup**". This will created the image manifest file and update the S3 input location path. Wait for "**Input data connection successful**"

<img src="../lab-images/completedatasetup.png">


### Additional Configuration
- Expand **Additional Configuration** 
- Validate that **Full dataset** is selected (This is used to specify whether you want to provide all the images to labeling job or a sub set of images based on filters or random sampling)

<img src="../lab-images/4.png" width="600">
<img src="../lab-images/5.png" width="600">

### Labeling Task 
- From the **Task type** drop down, select **Image**. Since you need to do annotation on images

<img src="../lab-images/6.png" width="600">

<img src="../lab-images/7.png" width="600">

### Task Selection
- This is Object Detection use case so you need to select **Bounding box** option  
- Leave other options as default and click on **Next** button

<img src="../lab-images/8.png" width="600">

### Select workers and configure tool
- Select **Private** in **Worker types**. For this lab, you will select an internal workforce to annotate the images. You have the option to select Public contractual workforce i.e **Amazon Mechanical Turk** or Partner workforce i.e. **Vendor managed** depending on your use case.
- In **Private teams** select the team name - **'labeling-experts'**

<img src="../lab-images/9.png" width="600"> 

### Labeling Instructions Template
- Leave other configurations default and scroll down to **Bounding box labeling tool**
- Add two labels as shown below - **'hole'** and **'no_hole'**
- Add detailed instructions in the **Description tab** for providing instructions to the workers - For example, you can specify - **You need to label woodpecker hole in the provided image. Please ensure that you select label 'hole' and draw the box around the woodpecker hole just to fit the hole for better quality of label data. You also need to label other areas which look similar to woodpecker holes but are not woodpecker holes with label 'no_hole'**
- You can also *optionally* provide examples of Good and Bad labeling image. You need to make sure that these images are publicly accessible.
- Click on **Create** button


<img src="../lab-images/10.png" width="500">

### Start Labeling Job
- Once you have successfully created the job, you will see that the **status** of the job is **"InProgress"**. This means that the job is created and the private workforce is notified via **email** regarding the task assigned to them. Since in this case, you have assigned the task to yourself. You should have received email with instructions to login to Ground Truth Labeling project
- **Open the email** and click on the **link** provided
- Enter **username** and **password** provided in the email. You may have to change the one time password provided in the email with a new password after login
- After you login, you will see the below screen
- Click on **Start working** button

<img src="../lab-images/11.png" width="700">

### Labeling Task
- You can use the provided tools to **Zoom in**, **Zoom out**, **Move** and **Box** sections in the images.
- You need to first select a **label** i.e. either **hole** or **no_hole** and then draw box in the image to annotate.
- Once you are annotating the required objects, click on **Submit** button

<img src="../lab-images/12.png" heigth="900">

### Complete Labeling Task
- You need to ensure that the bounding box is just enough to bound the object of interest
- Everytime you need to drsw bounding box, you need to first select the label on the right panel and then draw box around the object

<img src="../lab-images/13.png">

###  Check Labeling Job Status
A Ground Truth job can take a few hours to complete (if your dataset is larger than 10000 images, it can take much longer than that!). One way to monitor the job's progress is through AWS Console. In this notebook, we will use Ground Truth output files to monitor the progress.

You can re-evaluate the next cell repeatedly. It sends a `describe_labeling_job` request which should tell you whether the job is completed or not. If it is, then 'LabelingJobStatus' will be 'Completed'.

In [ ]:
job_name = 'aws-workshops-woodpecker-holes' 
sagemaker_client = boto3.client('sagemaker')
sagemaker_client.describe_labeling_job(LabelingJobName=job_name)['LabelingJobStatus']

### Inspect Labeled Data Sets

In [ ]:
job_url = "https://"+region+".console.aws.amazon.com/sagemaker/groundtruth?region="+region+"#/labeling-jobs/details/"+job_name
print (job_url)

### Labeled Data Sets
- Once you have labeled all the images, you will be taken to the SageMaker labeling project home page. This page shows you the **Labeled dataset** as shown below
- You can see how the different labels are applied. Now, training data for Amazon Rekognition Custom Labels is ready.

<img src="../lab-images/14.png">

# Review

We covered a lot of ground in this notebook! Let's recap what we accomplished. We uploaded images to S3 bucket and used SageMake Ground Truth labeling job to label the images and generated new labels for all of the images in our dataset. 